In [4]:
import sys
sys.path.append("..")

from sarimaCV import parallelize
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
import warnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tools.sm_exceptions import ConvergenceWarning

In [2]:
# Load the dataset
dataset = sm.datasets.co2.load_pandas()
co2_data = dataset.data

In [6]:
# Define a function that trains a single model over a single fold and add the @parallelize decorator with the desired parameters
@parallelize(n_folds=5, pred_len=1)
def eval_fold(data, p, d, q, P, D, Q, s):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', UserWarning)
        warnings.simplefilter('ignore', ConvergenceWarning)
        '''Evaluate a single fold of the cross-validation process'''
        train, test = data

        # Standard forecasting method
        model = SARIMAX(train, order=(p, d, q),
                        seasonal_order=(P, D, Q, s))
        forecasts = model.fit(disp=0).forecast(steps=len(test))

        sse = np.sum((test - np.array(forecasts))**2)
        return sse

In [9]:
res = eval_fold(
    data=co2_data["co2"],
    p=[0, 1],
    d=[0, 1],
    q=[0, 1],
    P=[0, 1],
    D=[1],
    Q=[0, 1],
    s=12
)
res.head()

,Error
ARIMA,
"(1, 1, 1, 1, 1, 1)",0.037990
"(1, 1, 1, 0, 1, 1)",0.039063
"(1, 1, 1, 1, 1, 0)",0.047991
"(1, 1, 0, 1, 1, 1)",0.070881
"(0, 1, 1, 1, 1, 1)",0.072467
